# 摩拜骑行数据清洗

## 初始及导入

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime as dt

# 计算订单距离
from geopy.distance import distance

# 获取文件现在的路径，用于文件储存
import os
os.getcwd()

'C:\\Users\\willl\\OneDrive - WillfortheFuture\\My Documents\\Study\\Python\\Udacity\\DA Advanced\\P5-Tableau'

In [2]:
mobike = pd.read_csv("mobike_shanghai_sample_updated.csv", low_memory=False)
mobike.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."


In [3]:
mobike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


## 整理时间序列列，并计算订单时间及轨迹点数量

In [4]:
# 将订单起止时间转换为datetime数据
mobike['start_time'] = pd.to_datetime(mobike['start_time'])
mobike['end_time'] = pd.to_datetime(mobike['end_time'])

In [5]:
# 计算订单用时
mobike['order_duration'] = mobike['end_time'] - mobike['start_time']

In [6]:
# 计算轨迹点数量
mobike['trackpoints'] = mobike['track'].str.count("\#")

In [7]:
mobike.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,order_duration,trackpoints
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",00:07:00,15
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",00:22:00,35
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",00:19:00,40
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",00:09:00,17
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",00:09:00,7


In [8]:
mobike.describe()

,orderid,bikeid,userid,start_location_x,start_location_y,end_location_x,end_location_y,order_duration,trackpoints
count,1.023610e+05,102361.000000,102361.000000,102361.000000,102361.000000,102361.000000,102361.000000,102361,102361.000000
mean,8.940838e+05,155425.868690,7834.435644,121.454144,31.251740,121.453736,31.252029,0 days 00:17:11.709733,22.261838
std,5.219651e+05,98816.625402,4530.387938,0.060862,0.057358,0.061577,0.057740,0 days 00:34:02.995113,21.783681
min,6.000000e+00,3.000000,1.000000,121.173000,30.842000,120.486000,30.841000,0 days 00:01:00,1.000000
25%,4.386790e+05,74701.000000,4095.000000,121.415000,31.212000,121.414000,31.212000,0 days 00:07:00,9.000000
50%,8.901510e+05,141990.000000,7617.000000,121.456000,31.260000,121.456000,31.261000,0 days 00:12:00,16.000000
75%,1.343252e+06,228303.000000,11278.000000,121.497000,31.294000,121.497000,31.294000,0 days 00:20:00,27.000000
max,1.807864e+06,387351.000000,17753.000000,121.970000,31.450000,121.971000,31.477000,3 days 06:45:00,478.000000


In [9]:
mobike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 12 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null datetime64[ns]
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null datetime64[ns]
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
order_duration      102361 non-null timedelta64[ns]
trackpoints         102361 non-null int64
dtypes: datetime64[ns](2), float64(4), int64(4), object(1), timedelta64[ns](1)
memory usage: 9.4+ MB


## 计算订单起点和终点的直线距离

In [10]:
# https://geopy.readthedocs.io/en/stable/#module-geopy.distance
# https://gis.stackexchange.com/questions/189945/using-geopys-vincenty-to-find-distances-in-pandas-data-frame
mobike['linear_distance'] = mobike.apply(lambda x: distance((x[5], x[4]), (x[8], x[7])).km, axis = 1)

In [11]:
mobike.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,order_duration,trackpoints,linear_distance
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",00:07:00,15,0.863185
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",00:22:00,35,2.015072
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",00:19:00,40,2.198627
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",00:09:00,17,1.355871
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",00:09:00,7,1.137383


In [12]:
mobike.describe()

,orderid,bikeid,userid,start_location_x,start_location_y,end_location_x,end_location_y,order_duration,trackpoints,linear_distance
count,1.023610e+05,102361.000000,102361.000000,102361.000000,102361.000000,102361.000000,102361.000000,102361,102361.000000,102361.000000
mean,8.940838e+05,155425.868690,7834.435644,121.454144,31.251740,121.453736,31.252029,0 days 00:17:11.709733,22.261838,1.564143
std,5.219651e+05,98816.625402,4530.387938,0.060862,0.057358,0.061577,0.057740,0 days 00:34:02.995113,21.783681,1.536494
min,6.000000e+00,3.000000,1.000000,121.173000,30.842000,120.486000,30.841000,0 days 00:01:00,1.000000,0.146068
25%,4.386790e+05,74701.000000,4095.000000,121.415000,31.212000,121.414000,31.212000,0 days 00:07:00,9.000000,0.723500
50%,8.901510e+05,141990.000000,7617.000000,121.456000,31.260000,121.456000,31.261000,0 days 00:12:00,16.000000,1.124994
75%,1.343252e+06,228303.000000,11278.000000,121.497000,31.294000,121.497000,31.294000,0 days 00:20:00,27.000000,1.883586
max,1.807864e+06,387351.000000,17753.000000,121.970000,31.450000,121.971000,31.477000,3 days 06:45:00,478.000000,95.603682


In [13]:
mobike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 13 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null datetime64[ns]
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null datetime64[ns]
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
order_duration      102361 non-null timedelta64[ns]
trackpoints         102361 non-null int64
linear_distance     102361 non-null float64
dtypes: datetime64[ns](2), float64(5), int64(4), object(1), timedelta64[ns](1)
memory usage: 10.2+ MB


## 获取用户的每日骑行情况

### 获取每个userid的每日订单情况

In [14]:
# code
user_freq = mobike.pivot_table(index=['userid', mobike['start_time'].dt.date],
                               values='orderid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)

user_freq = user_freq.reset_index()

In [15]:
# 记录用户首次下单情况：当首次记录到单车订单时，将其激活状态标定为True
user_freq['cummax'] = user_freq['orderid'].groupby(user_freq['userid']).cummax()
user_freq['activated'] = np.where(user_freq['cummax'] == 0, False, True)

# 去除辅助计算用的cummax列
user_freq = user_freq.drop(columns='cummax')

In [16]:
# test
user_freq.head()

,userid,start_time,orderid,activated
0,1,2016-08-01,0,False
1,1,2016-08-02,0,False
2,1,2016-08-03,0,False
3,1,2016-08-04,0,False
4,1,2016-08-05,0,False


### 获取用户每日使用次数（按userid计数）

In [17]:
# code
user_freq_pivot = user_freq[user_freq['activated'] == True].pivot_table(index=[user_freq['start_time'].dt.date, 'orderid'],
                               values='userid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)

user_freq_pivot = user_freq_pivot.reset_index()

In [18]:
# test
user_freq_pivot.head()

,start_time,orderid,userid
0,2016-08-01,0,0
1,2016-08-01,1,1316
2,2016-08-01,2,221
3,2016-08-01,3,19
4,2016-08-01,4,3


### 计算每位用户的月度活动情况

In [19]:
# code
user_activity = mobike.pivot_table(index=['userid'],
                               columns = mobike['start_time'].dt.date,
                               values='orderid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)

In [20]:
user_activity.head(1)

start_time,2016-08-01 00:00:00,2016-08-02 00:00:00,2016-08-03 00:00:00,2016-08-04 00:00:00,2016-08-05 00:00:00,2016-08-06 00:00:00,2016-08-07 00:00:00,2016-08-08 00:00:00,2016-08-09 00:00:00,2016-08-10 00:00:00,...,2016-08-22 00:00:00,2016-08-23 00:00:00,2016-08-24 00:00:00,2016-08-25 00:00:00,2016-08-26 00:00:00,2016-08-27 00:00:00,2016-08-28 00:00:00,2016-08-29 00:00:00,2016-08-30 00:00:00,2016-08-31 00:00:00
userid,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,2,1


In [21]:
user_activity['number_of_orders'] = user_activity.sum(axis=1)
user_activity['active_days'] = (user_activity != 0).sum(axis=1) - 1

In [22]:
user_activity = user_activity[['number_of_orders', 'active_days']].reset_index()
user_activity = user_activity.copy()

In [23]:
user_activity.head(5)

start_time,userid,number_of_orders,active_days
0,1,5,4
1,3,8,8
2,6,2,1
3,7,8,6
4,8,7,5


### 8月摩拜用户的活跃天数情况

In [25]:
user_active_days_pivot = user_activity.pivot_table(index='active_days', values='userid', aggfunc=len)
user_active_days_pivot = user_active_days_pivot.reset_index()

In [28]:
user_active_days_pivot

start_time,active_days,userid
0,1,1442
1,2,1966
2,3,2168
3,4,2301
4,5,2194
5,6,1977
6,7,1613
7,8,1216
8,9,855
9,10,543


### 8月摩拜用户的下单数量分布

In [29]:
user_order_counts_pivot = user_activity.pivot_table(index='number_of_orders', values='userid', aggfunc=len)
user_order_counts_pivot = user_order_counts_pivot.reset_index()

In [30]:
user_order_counts_pivot

start_time,number_of_orders,userid
0,1,1194
1,2,1565
2,3,1715
3,4,1857
4,5,1889
5,6,1768
6,7,1653
7,8,1360
8,9,1151
9,10,820


### 对用户增长的粗略估计

In [31]:
user_growth_e = user_freq[user_freq['activated'] == True].pivot_table(index=[user_freq['start_time'].dt.date],
                               values='userid',
                                aggfunc=max,
                               fill_value=0,
                               dropna=False)

user_growth_e = user_growth_e.reset_index()

user_growth_e['release_e'] =  user_growth_e['userid'].diff()

In [32]:
user_growth_e

,start_time,userid,release_e
0,2016-08-01,8605,NaN
1,2016-08-02,8663,58.0
2,2016-08-03,9324,661.0
3,2016-08-04,9714,390.0
4,2016-08-05,10336,622.0
5,2016-08-06,10548,212.0
6,2016-08-07,10888,340.0
7,2016-08-08,10954,66.0
8,2016-08-09,11298,344.0
9,2016-08-10,11865,567.0


## 获取每辆单车的每日使用情况

### 获取每辆单车每日承载单量情况

In [33]:
# code: 获取每辆单车每日承载量情况
bike_usage = mobike.pivot_table(index=['bikeid', mobike['start_time'].dt.date],
                               values='orderid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)
bike_usage = bike_usage.reset_index()

In [34]:
# 记录单车首次激活情况：当首次记录到单车订单时，将其激活状态标定为True
bike_usage['cummax'] = bike_usage['orderid'].groupby(bike_usage['bikeid']).cummax()
bike_usage['activated'] = np.where(bike_usage['cummax'] == 0, False, True)

# 去除辅助计算用的cummax列
bike_usage = bike_usage.drop(columns='cummax')

In [35]:
# test1
bike_usage.head()

,bikeid,start_time,orderid,activated
0,3,2016-08-01,0,False
1,3,2016-08-02,0,False
2,3,2016-08-03,0,False
3,3,2016-08-04,0,False
4,3,2016-08-05,0,False


In [36]:
bike_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450922 entries, 0 to 2450921
Data columns (total 4 columns):
bikeid        int64
start_time    datetime64[ns]
orderid       int64
activated     bool
dtypes: bool(1), datetime64[ns](1), int64(2)
memory usage: 58.4 MB


### 获取单车每日承载次数（按bikeid计数，只选择激活单车）

In [37]:
# code
bike_usage_pivot = bike_usage[bike_usage['activated'] == True].pivot_table(index=[bike_usage['start_time'].dt.date, 'orderid'],
                               values='bikeid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)

bike_usage_pivot = bike_usage_pivot.reset_index()

In [38]:
bike_usage_pivot.head()

,start_time,orderid,bikeid
0,2016-08-01,0,0
1,2016-08-01,1,1744
2,2016-08-01,2,40
3,2016-08-01,3,1
4,2016-08-02,0,1718


In [39]:
bike_usage_pivot.describe()

,orderid,bikeid
count,124.00000,124.000000
mean,1.50000,8794.669355
std,1.12257,17475.613142
min,0.00000,0.000000
25%,0.75000,4.000000
50%,1.50000,103.000000
75%,2.25000,4598.750000
max,3.00000,73780.000000


### 获取单车的8月活跃天数

In [40]:
# code
bike_activity = mobike.pivot_table(index=['bikeid'],
                               columns = mobike['start_time'].dt.date,
                               values='orderid',
                               aggfunc=len,
                               fill_value=0,
                               dropna=False)

In [41]:
bike_activity.head(1)

start_time,2016-08-01 00:00:00,2016-08-02 00:00:00,2016-08-03 00:00:00,2016-08-04 00:00:00,2016-08-05 00:00:00,2016-08-06 00:00:00,2016-08-07 00:00:00,2016-08-08 00:00:00,2016-08-09 00:00:00,2016-08-10 00:00:00,...,2016-08-22 00:00:00,2016-08-23 00:00:00,2016-08-24 00:00:00,2016-08-25 00:00:00,2016-08-26 00:00:00,2016-08-27 00:00:00,2016-08-28 00:00:00,2016-08-29 00:00:00,2016-08-30 00:00:00,2016-08-31 00:00:00
bikeid,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
bike_activity['number_of_orders'] = bike_activity.sum(axis=1)
bike_activity['active_days'] = (bike_activity != 0).sum(axis=1) - 1

In [43]:
bike_activity = bike_activity[['number_of_orders', 'active_days']].reset_index()

In [44]:
bike_activity.head(5)

start_time,bikeid,number_of_orders,active_days
0,3,1,1
1,6,1,1
2,13,2,2
3,22,1,1
4,25,1,1


### 8月单车的活跃天数情况

In [45]:
bike_active_days_pivot = bike_activity.pivot_table(index='active_days', values='bikeid', aggfunc=len)
bike_active_days_pivot = bike_active_days_pivot.reset_index()

In [46]:
bike_active_days_pivot

start_time,active_days,bikeid
0,1,63324
1,2,11806
2,3,2834
3,4,755
4,5,238
5,6,72
6,7,24
7,8,6
8,9,2
9,10,1


### 8月单车承载单量分布

In [47]:
bike_order_counts_pivot = bike_activity.pivot_table(index='number_of_orders', values='bikeid', aggfunc=len)
bike_order_counts_pivot = bike_order_counts_pivot.reset_index()

In [48]:
bike_order_counts_pivot

start_time,number_of_orders,bikeid
0,1,62155
1,2,12490
2,3,3113
3,4,862
4,5,294
5,6,91
6,7,41
7,8,10
8,9,5
9,11,1


### 对单车投放的粗略估计

In [49]:
bike_release_e = bike_usage[bike_usage['activated'] == True].pivot_table(index=[bike_usage['start_time'].dt.date],
                               values='bikeid',
                                aggfunc=max,
                               fill_value=0,
                               dropna=False)

bike_release_e = bike_release_e.reset_index()

bike_release_e['release_e'] =  bike_release_e['bikeid'].diff()

In [50]:
bike_release_e

,start_time,bikeid,release_e
0,2016-08-01,129627,NaN
1,2016-08-02,134312,4685.0
2,2016-08-03,139574,5262.0
3,2016-08-04,145425,5851.0
4,2016-08-05,151873,6448.0
5,2016-08-06,159055,7182.0
6,2016-08-07,167056,8001.0
7,2016-08-08,174507,7451.0
8,2016-08-09,180977,6470.0
9,2016-08-10,188646,7669.0


## 整理轨迹点

目的说明：我们将整理轨迹点信息，以在Tableau中绘制描述路径的蜘蛛图。现在我们有一串为字符串的轨迹点数据，与轨迹点数据有出入的订单起始点数据，以及轨迹点可能不能准确反映运动轨迹这一条信息。

需要做的事情：
- 判断订单实际方向和轨迹点方向是否一致
    - 为轨迹点数据单独新建数据集，命名为ordertrack
    - 判断实际订单方向
        - 根据订单起始点坐标，检验是否存在正南/北方向，正东/西向订单（不是北京，应该还好吧）
        - 根据订单起始点坐标，判断订单实际方向
    - 判断轨迹点方向
        - 提取实际轨迹的起始点坐标
        - 根据轨迹起始点坐标，判断轨迹点揭示的方向
    - 判断订单实际方向和轨迹点方向是否一致，生成判断列，以根据判断列分别采取措施
    - 提取所有轨迹点
    - 将轨迹点整理为以orderid为主键，按照Tableau绘制蜘蛛图要求的数据集格式
    
    
需要注意的有以下几点：
- 我们注意到，订单起始点和轨迹点有0.00x的差距。我们决定以轨迹点数据作为单车运行轨迹的唯一依据，以订单起始点作为参考，理由如下：
    - 轨迹点坐标和订单起始点坐标的采集方式可能不同：前者使用单车定位，后者可能基于手机定位；
    - 采集方式不同的假设可能能被两种数据的数据格式不同加强：轨迹点数据以字符串的形式出现，其更类似于日志，而非真正意义上的后台数据；
    - 运营后台获取用户手机定位坐标时，根据GPS，蓝牙和WIFI开启情况，以及周围遮蔽物情况，可能会产生与实际较大的误差。
- 我们实际上假设了单车轨迹点与实际的起始位置没有或仅有很小的差距，实际情况可能并非如此；工作场景中，应该需要询问相关负责人字段收集的方法进一步权衡；
- 我们完全没有考虑用户可能会走回头路的可能性，尽管这在现实生活中（至少是北京）十分常见。在北京主干道上想过街？先向北骑10分钟找红绿灯吧~
- 我们仅仅排除了根据订单起始点（我们认为有误差的手机定位）得出的，没有正南/北，正东/西向订单，实际结果可能略有出入；
- 因为排除了正东/西/南/北方向的订单，我们实际上将骑行方向的基本模型简化为了`东南`，`东北`，`西南`，`西北`四个。这么做的好处是，是否东西/南北这个问题，能用`是`或`否`来回答。这在现实生活中不现实，也意味着这一模型不能简单地套用到其他城市上。

### 为轨迹点数据新建单独数据集

In [51]:
# code: 新建单独数据集用于统计轨迹点信息
ordertrack = pd.concat([mobike['orderid'], 
                         mobike['start_location_x'], mobike['start_location_y'],
                         mobike['end_location_x'], mobike['end_location_y'], mobike['track']], axis = 1)


In [52]:
# test
ordertrack.head(1)

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track
0,78387,121.348,31.389,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."


### 判断实际订单方向

#### 判断是否存在正南/北，正东/西向订单

In [53]:
# code & test: 判断正东/西向订单。结论：没有
ordertrack[ordertrack['end_location_x'] == ordertrack['start_location_x']]

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track


In [54]:
# code & test: 判断正南/北向订单。结论：没有
ordertrack[ordertrack['end_location_y'] == ordertrack['start_location_y']]

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track


#### 根据起始坐标点，判断订单实际方向

In [55]:
# code: 计算订单实际方向
ordertrack['actual_E_W'] = np.where(ordertrack['end_location_x'] >= ordertrack['start_location_x'], 'East', 'West')
ordertrack['actual_N_S'] = np.where(ordertrack['end_location_y'] >= ordertrack['start_location_y'], 'North', 'South')
ordertrack['actual_direction'] = ordertrack['actual_N_S'] + ordertrack['actual_E_W']
ordertrack = ordertrack.drop(columns = ['actual_E_W', 'actual_N_S'])

In [56]:
# test
ordertrack.head(1)

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track,actual_direction
0,78387,121.348,31.389,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",SouthEast


### 判断轨迹点方向

#### 提取实际轨迹的起始点坐标

In [57]:
# code1: 提取轨迹点的起始点信息

# 提取track里的起始坐标点经纬度信息
temp_df1 = ordertrack['track'].str.split("#", n=1, expand=True)
ordertrack['first_x'] = temp_df1[0].str.split(",", n=1, expand=True)[0]
ordertrack['first_y'] = temp_df1[0].str.split(",", n=1, expand=True)[1]

# 提取track里的最后坐标点经纬度信息
temp_df2 = ordertrack['track'].str.rsplit("#", n=1, expand=True) # 首先提取最后一组坐标点并赋值给一个临时数据集
ordertrack['last_x'] = temp_df2[1].str.split(",", n=1, expand=True)[0] # 在最后一组坐标点中分别提取经度和纬度，并重新赋值给临时数据集
ordertrack['last_y'] = temp_df2[1].str.split(",", n=1, expand=True)[1] 

In [58]:
# code2: 将提取成功的起始点坐标信息
ordertrack['first_x'] = ordertrack['first_x'].astype('float', copy=False)
ordertrack['first_y'] = ordertrack['first_y'].astype('float', copy=False)
ordertrack['last_x'] = ordertrack['last_x'].astype('float', copy=False)
ordertrack['last_y'] = ordertrack['last_y'].astype('float', copy=False)

In [59]:
# test
ordertrack.head()

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track,actual_direction,first_x,first_y,last_x,last_y
0,78387,121.348,31.389,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",SouthEast,121.347,31.392,121.357,31.389
1,891333,121.508,31.279,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",SouthWest,121.489,31.270,121.508,31.279
2,1106623,121.383,31.254,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",SouthEast,121.381,31.251,121.407,31.249
3,1389484,121.484,31.320,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",NorthWest,121.471,31.325,121.484,31.320
4,188537,121.407,31.292,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",SouthEast,121.407,31.291,121.418,31.288


#### 根据轨迹起始点坐标，判断轨迹排列方向

In [60]:
# code: 计算轨迹点方向
ordertrack['track_E_W'] = np.where(ordertrack['last_x'] >= ordertrack['first_x'], 'East', 'West')
ordertrack['track_N_S'] = np.where(ordertrack['last_y'] >= ordertrack['first_y'], 'North', 'South')
ordertrack['track_direction'] = ordertrack['track_N_S'] + ordertrack['track_E_W']
ordertrack = ordertrack.drop(columns = ['track_E_W', 'track_N_S'])

In [61]:
# test
ordertrack.head()

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,track,actual_direction,first_x,first_y,last_x,last_y,track_direction
0,78387,121.348,31.389,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",SouthEast,121.347,31.392,121.357,31.389,SouthEast
1,891333,121.508,31.279,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",SouthWest,121.489,31.270,121.508,31.279,NorthEast
2,1106623,121.383,31.254,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1...",SouthEast,121.381,31.251,121.407,31.249,SouthEast
3,1389484,121.484,31.320,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1...",NorthWest,121.471,31.325,121.484,31.320,SouthEast
4,188537,121.407,31.292,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1...",SouthEast,121.407,31.291,121.418,31.288,SouthEast


#### 判断订单方向与轨迹点方向的一致性

In [62]:
# code: 去除不需要的列
ordertrack1 = ordertrack.drop(columns = ['start_location_x', 'start_location_y',
                                       'end_location_x', 'end_location_y',
                                       'first_x', 'first_y', 'last_x', 'last_y'])

ordertrack1 = ordertrack1.copy()

In [63]:
# code: 计算实际与轨迹方向是否相符
ordertrack1['direction_type'] = np.where(ordertrack1['actual_direction'] == ordertrack1['track_direction'], 'match', 'mismatch')
ordertrack1 = ordertrack1.drop(columns = ['actual_direction', 'track_direction'])

In [64]:
# test
ordertrack1.head()

,orderid,track,direction_type
0,78387,"121.347,31.392#121.348,31.389#121.349,31.390#1...",match
1,891333,"121.489,31.270#121.489,31.271#121.490,31.270#1...",mismatch
2,1106623,"121.381,31.251#121.382,31.251#121.382,31.252#1...",match
3,1389484,"121.471,31.325#121.472,31.325#121.473,31.324#1...",mismatch
4,188537,"121.407,31.291#121.407,31.292#121.408,31.291#1...",match


### 提取所有轨迹点

In [65]:
# code: 提取所有的轨迹点，并为轨迹点附上orderid
ordertrack2 = ordertrack1['track'].str.split("#", n=-1, expand=True)
ordertrack2['orderid'] = ordertrack1['orderid']
ordertrack2['direction_type'] = ordertrack1['direction_type']

In [66]:
# test
ordertrack2.head()

,0,1,2,3,4,5,6,7,8,9,...,471,472,473,474,475,476,477,478,orderid,direction_type
0,"121.347,31.392","121.348,31.389","121.349,31.390","121.350,31.390","121.351,31.390","121.351,31.391","121.352,31.391","121.353,31.391","121.353,31.394","121.354,31.391",...,None,None,None,None,None,None,None,None,78387,match
1,"121.489,31.270","121.489,31.271","121.490,31.270","121.490,31.271","121.491,31.271","121.492,31.270","121.492,31.271","121.493,31.269","121.493,31.270","121.494,31.268",...,None,None,None,None,None,None,None,None,891333,mismatch
2,"121.381,31.251","121.382,31.251","121.382,31.252","121.382,31.253","121.382,31.254","121.383,31.250","121.383,31.251","121.383,31.254","121.384,31.250","121.385,31.250",...,None,None,None,None,None,None,None,None,1106623,match
3,"121.471,31.325","121.472,31.325","121.473,31.324","121.473,31.325","121.474,31.322","121.474,31.323","121.475,31.320","121.475,31.321","121.475,31.322","121.476,31.320",...,None,None,None,None,None,None,None,None,1389484,mismatch
4,"121.407,31.291","121.407,31.292","121.408,31.291","121.411,31.288","121.414,31.288","121.417,31.288","121.418,31.287","121.418,31.288",None,None,...,None,None,None,None,None,None,None,None,188537,match


### 将轨迹点整理为以orderid为主键，符合Tableau蜘蛛图绘图要求的格式

#### 准备待溶解的数据集

In [67]:
# code1
ordertrack_match = ordertrack2[ordertrack2['direction_type'] == 'match']
ordertrack_match = ordertrack_match.drop(columns = 'direction_type')

In [68]:
# test1
ordertrack_match.head(1)

,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,orderid
0,"121.347,31.392","121.348,31.389","121.349,31.390","121.350,31.390","121.351,31.390","121.351,31.391","121.352,31.391","121.353,31.391","121.353,31.394","121.354,31.391",...,None,None,None,None,None,None,None,None,None,78387


In [69]:
# code2
ordertrack_mismatch = ordertrack2[ordertrack2['direction_type'] == 'mismatch']
ordertrack_mismatch = ordertrack_mismatch.drop(columns = 'direction_type')

In [70]:
# test1
ordertrack_mismatch.head(1)

,0,1,2,3,4,5,6,7,8,9,...,470,471,472,473,474,475,476,477,478,orderid
1,"121.489,31.270","121.489,31.271","121.490,31.270","121.490,31.271","121.491,31.271","121.492,31.270","121.492,31.271","121.493,31.269","121.493,31.270","121.494,31.268",...,None,None,None,None,None,None,None,None,None,891333


#### 分别溶解数据集并排序

##### 顺序坐标点

In [71]:
# code: 溶解轨迹与实际方向一致的数据集，并清空空值
# Tableau要求格式见：https://onlinehelp.tableau.com/current/pro/desktop/zh-cn/maps_howto_origin_destination.htm#Order
ordertrack_match_melted = ordertrack_match.melt(id_vars='orderid', var_name = 'point_order', value_name = 'track_points')
ordertrack_match_melted = ordertrack_match_melted.dropna()

In [72]:
# code: 进一步分离坐标点以获得经纬度信息
ordertrack_match_melted['track_x'] = ordertrack_match_melted['track_points'].str.split(",", n=1, expand=True)[0]
ordertrack_match_melted['track_y'] = ordertrack_match_melted['track_points'].str.split(",", n=1, expand=True)[1]

# 将作为为本提取的经纬度信息
ordertrack_match_melted['track_x'] = ordertrack_match_melted['track_x'].astype('float', copy=False)
ordertrack_match_melted['track_y'] = ordertrack_match_melted['track_y'].astype('float', copy=False)

In [73]:
# beautify
ordertrack_match_melted = ordertrack_match_melted.sort_values(by=['orderid', 'point_order'], axis = 0)
ordertrack_match_melted = ordertrack_match_melted.drop(columns='track_points')

In [74]:
# code：列名称重新排序
# https://github.com/pandas-dev/pandas/issues/21554
# https://blog.csdn.net/qq_22238533/article/details/72395564
ordertrack_match_melted['point_order'] = ordertrack_match_melted['track_x'].groupby(ordertrack_match_melted['orderid']).rank(method='first', ascending=True)

In [75]:
# beautify: 将float转化为整数
ordertrack_match_melted['point_order'] = ordertrack_match_melted['point_order'].astype('int', copy=False)

In [76]:
# test1
ordertrack_match_melted.head()

,orderid,point_order,track_x,track_y
36185,113,1,121.486,31.248
79688,113,2,121.486,31.249
123191,113,3,121.486,31.250
166694,113,4,121.487,31.250
210197,113,5,121.487,31.251


In [77]:
# test 2
ordertrack_match_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042877 entries, 36185 to 1282411
Data columns (total 4 columns):
orderid        1042877 non-null int64
point_order    1042877 non-null int32
track_x        1042877 non-null float64
track_y        1042877 non-null float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 75.8 MB


##### 倒序坐标点

In [78]:
# code: 溶解轨迹与实际方向一致的数据集，并清空空值
# Tableau要求格式见：https://onlinehelp.tableau.com/current/pro/desktop/zh-cn/maps_howto_origin_destination.htm#Order
ordertrack_mismatch_melted = ordertrack_mismatch.melt(id_vars='orderid', var_name = 'point_order', value_name = 'track_points')
ordertrack_mismatch_melted = ordertrack_mismatch_melted.dropna()

In [79]:
# code: 进一步分离坐标点以获得经纬度信息
ordertrack_mismatch_melted['track_x'] = ordertrack_mismatch_melted['track_points'].str.split(",", n=1, expand=True)[0]
ordertrack_mismatch_melted['track_y'] = ordertrack_mismatch_melted['track_points'].str.split(",", n=1, expand=True)[1]

In [80]:
# code: 调查错误的输入
ordertrack_mismatch_melted[ordertrack_mismatch_melted['track_y'].str.contains("\\", regex=False)]

,orderid,point_order,track_points,track_x,track_y
92417,841818,1,"121.494,31.26\",121.494,31.26\
105599,1447041,1,"121.485,31.28\",121.485,31.28\
463156,1558212,7,"121.427,31.29\",121.427,31.29\


In [81]:
# code: 调查错误的输入1
# 看上去是一个未写入完整的数据。我们用31.260替换，手动修改一下
ordertrack_mismatch_melted[ordertrack_mismatch_melted['orderid'] == 841818]

,orderid,point_order,track_points,track_x,track_y
33559,841818,0,"121.494,31.260",121.494,31.260
92417,841818,1,"121.494,31.26\",121.494,31.26\
151275,841818,2,"121.495,31.259",121.495,31.259
210133,841818,3,"121.495,31.260",121.495,31.260
268991,841818,4,"121.496,31.259",121.496,31.259


In [82]:
# code: 调查错误的输入2
# 看上去是一个未写入完整的数据。我们用31.280替换，手动修改一下
ordertrack_mismatch_melted[ordertrack_mismatch_melted['orderid'] == 1447041]

,orderid,point_order,track_points,track_x,track_y
46741,1447041,0,"121.485,31.279",121.485,31.279
105599,1447041,1,"121.485,31.28\",121.485,31.28\
164457,1447041,2,"121.486,31.279",121.486,31.279
223315,1447041,3,"121.486,31.282",121.486,31.282
282173,1447041,4,"121.487,31.279",121.487,31.279
341031,1447041,5,"121.487,31.282",121.487,31.282
399889,1447041,6,"121.488,31.279",121.488,31.279
458747,1447041,7,"121.489,31.279",121.489,31.279
517605,1447041,8,"121.490,31.279",121.490,31.279
576463,1447041,9,"121.490,31.280",121.490,31.280


In [83]:
# code: 调查错误的输入3
# 看上去是一个未写入完整的数据。我们用31.290替换，手动修改一下
ordertrack_mismatch_melted[ordertrack_mismatch_melted['orderid'] == 1558212].head(10)

,orderid,point_order,track_points,track_x,track_y
51150,1558212,0,"121.427,31.285",121.427,31.285
110008,1558212,1,"121.427,31.286",121.427,31.286
168866,1558212,2,"121.427,31.287",121.427,31.287
227724,1558212,3,"121.427,31.288",121.427,31.288
286582,1558212,4,"121.427,31.289",121.427,31.289
345440,1558212,5,"121.427,31.29",121.427,31.29
404298,1558212,6,"121.427,31.290",121.427,31.290
463156,1558212,7,"121.427,31.29\",121.427,31.29\
522014,1558212,8,"121.428,31.282",121.428,31.282
580872,1558212,9,"121.428,31.283",121.428,31.283


In [84]:
# code: 清理错误的输入
ordertrack_mismatch_melted.loc[92417, 'track_y'] = 31.260
ordertrack_mismatch_melted.loc[105599, 'track_y'] = 31.280
ordertrack_mismatch_melted.loc[463156, 'track_y'] = 31.290

In [85]:
# 将作为为本提取的经纬度信息
ordertrack_mismatch_melted['track_x'] = ordertrack_mismatch_melted['track_x'].astype('float', copy=False)
ordertrack_mismatch_melted['track_y'] = ordertrack_mismatch_melted['track_y'].astype('float', copy=False)

In [86]:
# beautify
ordertrack_mismatch_melted = ordertrack_mismatch_melted.sort_values(by=['orderid', 'point_order'], axis=0, ascending=False)
ordertrack_mismatch_melted = ordertrack_mismatch_melted.drop(columns='track_points')

In [87]:
# code：列名称重新排序
# https://github.com/pandas-dev/pandas/issues/21554
# https://blog.csdn.net/qq_22238533/article/details/72395564
ordertrack_mismatch_melted['point_order'] = ordertrack_mismatch_melted['track_x'].groupby(ordertrack_mismatch_melted['orderid']).rank(method='first', ascending=False)

In [88]:
# beautify: 将float转化为整数
ordertrack_mismatch_melted['point_order'] = ordertrack_mismatch_melted['point_order'].astype('int', copy=False)

In [89]:
# test1
ordertrack_mismatch_melted.head()

,orderid,point_order,track_x,track_y
824853,1807855,1,121.386,31.315
765995,1807855,2,121.385,31.315
707137,1807855,3,121.385,31.314
648279,1807855,4,121.384,31.314
589421,1807855,5,121.383,31.314


In [90]:
# test 2
ordertrack_mismatch_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338228 entries, 824853 to 8011
Data columns (total 4 columns):
orderid        1338228 non-null int64
point_order    1338228 non-null int32
track_x        1338228 non-null float64
track_y        1338228 non-null float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 85.9 MB


### 将整理完毕的路径点数据集合并

In [91]:
# code: 将溶解完毕的数据集整合为独立的路径点数据集waypoint
waypoint = pd.concat([ordertrack_match_melted, ordertrack_mismatch_melted], ignore_index=True)

In [92]:
# test 1
waypoint.sample(5)

,orderid,point_order,track_x,track_y
925707,1619954,6,121.412,31.229
651782,1170176,6,121.428,31.193
604197,1086030,24,121.385,31.289
1859507,776231,29,121.460,31.245
1205304,1620812,16,121.413,31.303


In [93]:
# test 2
waypoint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2381105 entries, 0 to 2381104
Data columns (total 4 columns):
orderid        int64
point_order    int32
track_x        float64
track_y        float64
dtypes: float64(2), int32(1), int64(1)
memory usage: 63.6 MB


## 计算订单实际移动距离（经纬度）

要计算用户实际移动的距离，我们需要分别计算每个轨迹点之间的距离，最后再汇总。

考虑到需要频繁计算两个坐标之间的差值，而geopy包似乎没有提供类似于`pandas.diff`那样简便的计算数据集阶差的方式，我们需要对数据集进行变形，以符合`geopy`中`distance`函数要求的两个坐标的输入形式。与此同时，我们还需要注意，上一个订单的最后一个点和下一个订单的第一个点之间不能计算距离。

因此，我们确定如下思路：
- 首先复制waypoint数据集，命名为waypoint_diff，作为主数据集；
- 复制waypoint_diff数据集做临时数据集，命名为waypoint_diff1，并进行如下处理，得到用于就按坐标距离的数据集waypoint_diff2：
    - 去除index为0的项目，重设index（这样原来数据集的第二行就变成新数据集的第一行了），并删除`orderid`和原来的`index`列，仅保留坐标和point_order列；
    - 将waypoint_diff和waypoint_diff1数据集按index合并，右侧数据集以“2”作为标注，形成坐标点交错方便计算距离的新数据集waypoint_diff2；
    - 将waypoint_diff2中，point_order2列中值为1的数据全部改为np.nan；这些行里上一订单的最后一个点与下一订单的第一个点交错，需要去掉；
    - 利用`dropna()`方法去掉上述行
- 对waypoint_diff2数据集使用distance方法计算坐标点之间的距离，并保存在新列`dist`中
- 使用透视表，对dist列，按照orderid进行加和计算，将计算结果保存为`actual_order_distance`，重设index一重新利用orderid字段
- 将`actual_order_distance`合并至`mobike`数据集，以`orderid`为主键

### 计算每个订单每个轨迹点之间的距离差，及其前期整理工作

In [94]:
# code: 创建主数据集
waypoint_diff = waypoint.copy()

In [95]:
# test
waypoint_diff.iloc[96:101, :]

,orderid,point_order,track_x,track_y
96,113,97,121.553,31.277
97,113,98,121.554,31.277
98,113,99,121.555,31.277
99,130,1,121.531,31.297
100,130,2,121.531,31.298


In [96]:
# code: 创建辅助数据集
waypoint_diff1 = waypoint_diff.drop(index=0)
waypoint_diff1 = waypoint_diff1.reset_index()
waypoint_diff1 = waypoint_diff1.drop(columns=['index', 'orderid'])

In [97]:
# test
waypoint_diff1.head()

,point_order,track_x,track_y
0,2,121.486,31.249
1,3,121.486,31.250
2,4,121.487,31.250
3,5,121.487,31.251
4,6,121.488,31.251


In [98]:
# code: 创建计算数据集
waypoint_diff2 = waypoint_diff.join(waypoint_diff1, rsuffix='2')

# code: 整理计算数据集：去除不同订单坐标点衔接的行
waypoint_diff2['point_order2'] = waypoint_diff2['point_order2'].where(waypoint_diff2['point_order2'] != 1, np.nan)
waypoint_diff2 = waypoint_diff2.dropna()
waypoint_diff2 = waypoint_diff2.copy()

In [99]:
# test
waypoint_diff2.iloc[96:101, :]

,orderid,point_order,track_x,track_y,point_order2,track_x2,track_y2
96,113,97,121.553,31.277,98.0,121.554,31.277
97,113,98,121.554,31.277,99.0,121.555,31.277
99,130,1,121.531,31.297,2.0,121.531,31.298
100,130,2,121.531,31.298,3.0,121.531,31.299
101,130,3,121.531,31.299,4.0,121.531,31.300


In [100]:
# 计算坐标点之间的距离
waypoint_diff2['dist'] = waypoint_diff2.apply(lambda x: distance((x[3], x[2]), (x[6], x[5])).km, axis = 1)

In [101]:
waypoint_diff2.head()

,orderid,point_order,track_x,track_y,point_order2,track_x2,track_y2,dist
0,113,1,121.486,31.248,2.0,121.486,31.249,0.110874
1,113,2,121.486,31.249,3.0,121.486,31.250,0.110874
2,113,3,121.486,31.250,4.0,121.487,31.250,0.095254
3,113,4,121.487,31.250,5.0,121.487,31.251,0.110874
4,113,5,121.487,31.251,6.0,121.488,31.251,0.095253


### 计算每个订单的实际骑行距离

In [102]:
# code
actual_order_distance = waypoint_diff2.pivot_table(index='orderid', values='dist', aggfunc=sum).reset_index()

In [103]:
# test 1
actual_order_distance.head(5)

,orderid,dist
0,6,1.034212
1,24,13.989063
2,63,8.195751
3,68,10.798527
4,72,65.760991


In [104]:
# test 2
actual_order_distance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 2 columns):
orderid    102361 non-null int64
dist       102361 non-null float64
dtypes: float64(1), int64(1)
memory usage: 1.6 MB


### 将订单的实际骑行距离保存至`mobike`数据集中，并去除track列

In [105]:
# code
mobike = pd.merge(mobike, actual_order_distance, on='orderid')
mobike = mobike.drop(columns="track")

In [106]:
# test
mobike.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,order_duration,trackpoints,linear_distance,dist
0,78387,158357,10080,2016-08-20 06:57:00,121.348,31.389,2016-08-20 07:04:00,121.357,31.388,00:07:00,15,0.863185,2.676949
1,891333,92776,6605,2016-08-29 19:09:00,121.508,31.279,2016-08-29 19:31:00,121.489,31.271,00:22:00,35,2.015072,4.130218
2,1106623,152045,8876,2016-08-13 16:17:00,121.383,31.254,2016-08-13 16:36:00,121.405,31.248,00:19:00,40,2.198627,5.824772
3,1389484,196259,10648,2016-08-23 21:34:00,121.484,31.320,2016-08-23 21:43:00,121.471,31.325,00:09:00,17,1.355871,2.379540
4,188537,78208,11735,2016-08-16 07:32:00,121.407,31.292,2016-08-16 07:41:00,121.418,31.288,00:09:00,7,1.137383,1.523780


## 将已经生成的数据集保存为csv

In [107]:
mobike.to_csv('py_mobike_master.csv', na_rep='Null', header=True, index=False)



bike_usage.to_csv('py_bike_usage.csv', na_rep='Null', header=True, index=False)
bike_usage_pivot.to_csv('py_bike_usage_pivot.csv', na_rep='Null', header=True, index=False)
bike_active_days_pivot.to_csv('py_bike_active_days_pivot.csv', na_rep='Null', header=True, index=False)
bike_order_counts_pivot.to_csv('py_bike_order_counts_pivot.csv', na_rep='Null', header=True, index=False)
bike_release_e.to_csv('py_bike_release_e.csv', na_rep='Null', header=True, index=False)

user_freq.to_csv('py_user_freq.csv', na_rep='Null', header=True, index=False)
user_freq_pivot.to_csv('py_user_freq_pivot.csv', na_rep='Null', header=True, index=False)
user_active_days_pivot.to_csv('py_user_active_days_pivot.csv', na_rep='Null', header=True, index=False)
user_order_counts_pivot.to_csv('py_user_order_counts_pivot.csv', na_rep='Null', header=True, index=False)
user_growth_e.to_csv('py_user_growth_e.csv', na_rep='Null', header=True, index=False)

waypoint.to_csv('py_waypoint.csv', na_rep='Null', header=True, index=False)

## 获取订单起始点行政区划信息

### 为行政区划信息创建一个独立的，仅包含orderid和起始点坐标的数据集order_location

In [108]:
# code
order_location = ordertrack.drop(columns=['start_location_x', 'start_location_y',
                                         'end_location_x', 'end_location_y',
                                         'track', 'actual_direction', 'track_direction'])
order_location = order_location.copy()

In [109]:
order_location.to_csv('py_order_location_unfinished.csv', na_rep=np.nan, header=True, index=False)

### 读取经过地理逆编码得数据集，命名为order_location_s

In [112]:
order_location_s = pd.read_csv('py_order_location_s.csv')

In [113]:
order_location_s.head()

,orderid,first_x,first_y,last_x,last_y,start_loc,end_loc
0,78387,121.347,31.392,121.357,31.389,上海市宝山区美安路,上海市宝山区金中路
1,891333,121.489,31.270,121.508,31.279,上海市虹口区四川北路2075号,上海市虹口区阜新路180弄1号西
2,1106623,121.381,31.251,121.407,31.249,上海市嘉定区吉镇路,上海市普陀区杨柳青路
3,1389484,121.471,31.325,121.484,31.320,上海市闸北区阳泉路693号,上海市虹口区安汾路588号
4,188537,121.407,31.291,121.418,31.288,上海市普陀区汶水路,上海市宝山区真华路1376号


In [160]:
# 提取行政区划信息
order_location_s['start_district'] = order_location_s['start_loc'].str.extract(r'上海市(.{2,3}区)')
order_location_s['end_district'] = order_location_s['end_loc'].str.extract(r'上海市(.{2,3}区)')

# test
order_location_s.head()

,orderid,first_x,first_y,last_x,last_y,start_loc,end_loc,start_district,end_district,order_type,start_time
0,78387,121.347,31.392,121.357,31.389,上海市宝山区美安路,上海市宝山区金中路,宝山区,宝山区,同区,2016-08-20 06:57:00
1,891333,121.489,31.270,121.508,31.279,上海市虹口区四川北路2075号,上海市虹口区阜新路180弄1号西,虹口区,虹口区,同区,2016-08-29 19:09:00
2,1106623,121.381,31.251,121.407,31.249,上海市嘉定区吉镇路,上海市普陀区杨柳青路,嘉定区,普陀区,跨区,2016-08-13 16:17:00
3,1389484,121.471,31.325,121.484,31.320,上海市闸北区阳泉路693号,上海市虹口区安汾路588号,闸北区,虹口区,跨区,2016-08-23 21:34:00
4,188537,121.407,31.291,121.418,31.288,上海市普陀区汶水路,上海市宝山区真华路1376号,普陀区,宝山区,跨区,2016-08-16 07:32:00


In [161]:
order_location_s['start_district'].value_counts()

杨浦区     14399
普陀区     13061
徐汇区     10966
虹口区     10903
闸北区     10133
宝山区      9148
闵行区      7831
长宁区      7415
浦东新区     5424
嘉定区      4651
黄浦区      4626
静安区      2477
松江区       943
青浦区       369
奉贤区        14
Name: start_district, dtype: int64

In [162]:
# 判断订单类型
order_location_s['order_type'] = np.where(order_location_s['start_district'] == order_location_s['end_district'], '同区', '跨区')

In [163]:
# 为订单添加时间
order_location_s['start_time'] = mobike['start_time']

In [164]:
# test
order_location_s.head()

,orderid,first_x,first_y,last_x,last_y,start_loc,end_loc,start_district,end_district,order_type,start_time
0,78387,121.347,31.392,121.357,31.389,上海市宝山区美安路,上海市宝山区金中路,宝山区,宝山区,同区,2016-08-20 06:57:00
1,891333,121.489,31.270,121.508,31.279,上海市虹口区四川北路2075号,上海市虹口区阜新路180弄1号西,虹口区,虹口区,同区,2016-08-29 19:09:00
2,1106623,121.381,31.251,121.407,31.249,上海市嘉定区吉镇路,上海市普陀区杨柳青路,嘉定区,普陀区,跨区,2016-08-13 16:17:00
3,1389484,121.471,31.325,121.484,31.320,上海市闸北区阳泉路693号,上海市虹口区安汾路588号,闸北区,虹口区,跨区,2016-08-23 21:34:00
4,188537,121.407,31.291,121.418,31.288,上海市普陀区汶水路,上海市宝山区真华路1376号,普陀区,宝山区,跨区,2016-08-16 07:32:00


In [165]:
# 重新保存数据集
order_location_s.to_csv('order_location_s_final.csv',  na_rep='Null', header=True, index=False)